# 2010 BRFSS Data Cleaning

In [2]:
#import necessary packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier

## First stage data cleaning

**Process data so that:**
- All ACE-related questions are filled with either a "yes" or a "no"
- There are only variables that I am interested in including according to my inclusion/exclusion criteria
- All indices are correct

In [3]:
#keeping this seperate to have access to the raw, unprocessed data as needed
raw_data = pd.read_csv("/Users/elianasullivan/Documents/Minerva/last one/first semester/capstone/data/BRFSS_full_v4.csv")
data_1 = pd.DataFrame(data=raw_data)

/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (11,18,19,20,22,23,24,25,26,28,29,30,31,32,33,34,35,36,38,39,40,41,43,44,45,49,50,51,52,54,55,56,57,58,59,60,61,62,63,65,66,72,77,79,81,84,108,109,148,149,150,151,167,168,169,312,348,349,350,351,352,353,354,355,389,390,391,392,393,394,395,396) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
#convert the data from a csv on my computer to a data frame that I can work with
data = pd.DataFrame(data=raw_data)

In [5]:
#see a snapshot of the data
data.head()

,@_STATE,@_GEOSTR,@_DENSTR2,PRECALL,REPNUM,REPDEPTH,FMONTH,IDATE,IMONTH,IDAY,...,@_AIDTST2,CPCOUNTY,@_ITSCF1,@_ITSCF2,@_ITSPOST,@_ITSFINL,@_RAWHH,@_WT2HH,@_POSTHH,@_HOUSEWT
0,1,9,2,1,120127,25,12,12302010,12,30,...,,CLARKE3,3.35114e+06,88.7997,0.547504,48.6182,0.333333,18.0124,5.47287,98.5797
1,1,13,1,1,10170,26,1,2022010,2,2,...,,colbert,3.35114e+06,326.702,0.745529,243.566,1,33.1346,5.47399,181.379
2,1,17,2,1,80197,23,8,8182010,8,18,...,,housaton,3.35114e+06,189.48,1.24238,235.406,1,38.4347,5.80618,223.159
3,1,4,1,1,100041,19,10,11012010,11,1,...,,jefferson,3.35114e+06,931.521,0.883034,822.565,1,94.4764,5.27059,497.947
4,1,13,1,1,120172,28,12,12302010,12,30,...,,lauderdale,3.35114e+06,163.351,0.702397,114.737,1,33.1346,5.47399,181.379


In [6]:
#length of the full dataset before anything is removed
len(data)

451075

In [5]:
#the empty space isn't registering as the empty by looking at the "head" table 
#so this just takes a cell that I know is empty and makes that value for empty
empty = ((data["@_AIDTST2"][0]))
print(empty)

In [6]:
#for each of the ACE questions remove any rows that have an 'empty' value for any of the ACE questions
#prints how many are left after each ACE to see how many are datapoints are getting removed
aces = ["ACEDEPRS","ACEDRINK","ACEDRUGS","ACEPRISN","ACEDIVRC","ACEPUNCH","ACEHURT","ACESWEAR","ACETOUCH","ACETTHEM","ACEHVSEX"]

for exp in aces:
    data[exp].replace(empty, np.nan, inplace=True)
    all_filled = data.dropna(axis = 0, how='any')
    print(exp,len(all_filled))

('ACEDEPRS', 24405)
('ACEDRINK', 24380)
('ACEDRUGS', 24375)
('ACEPRISN', 24373)
('ACEDIVRC', 24367)
('ACEPUNCH', 24361)
('ACEHURT', 24352)
('ACESWEAR', 24342)
('ACETOUCH', 24334)
('ACETTHEM', 24328)
('ACEHVSEX', 24320)


In [7]:
#look at a preview of the updated data set
all_filled.head()

,@_STATE,@_GEOSTR,@_DENSTR2,PRECALL,REPNUM,REPDEPTH,FMONTH,IDATE,IMONTH,IDAY,...,@_AIDTST2,CPCOUNTY,@_ITSCF1,@_ITSCF2,@_ITSPOST,@_ITSFINL,@_RAWHH,@_WT2HH,@_POSTHH,@_HOUSEWT
59878,11,1,1,1,20002,19,2,3012010,3,1,...,1,,455432,127.697,0.516679,65.9785,1,6.87392,9.36442,64.3703
59879,11,1,2,1,20002,27,2,2272010,2,27,...,,,455432,95.7847,0.716141,68.5953,1,10.3122,9.36442,96.5673
59881,11,1,1,1,20003,30,2,3092010,3,9,...,2,,455432,127.697,1.10438,141.026,1,6.87392,9.36442,64.3703
59882,11,1,1,1,20005,7,2,3022010,3,2,...,1,,455432,63.8486,0.516679,32.9893,1,6.87392,9.36442,64.3703
59883,11,1,1,1,20005,15,2,2272010,2,27,...,,,455432,127.697,0.364598,46.5581,1,6.87392,9.36442,64.3703


In [8]:
#for each of the ACEs remove any rows for which people answered "I don't know" (7) or "refused" (9)
#this can be combined with the section above, but this helps to see how many more are lost to each of the conditions
aces = ["ACEDEPRS","ACEDRINK","ACEDRUGS","ACEPRISN","ACEDIVRC","ACEPUNCH","ACEHURT","ACESWEAR","ACETOUCH","ACETTHEM","ACEHVSEX"]

for exp in aces:
    all_filled[exp].replace("7", np.nan, inplace=True)
    all_filled[exp].replace("9", np.nan, inplace=True)
    all_clean = all_filled.dropna(axis = 0, how='any')
    print(exp,len(all_clean))

/anaconda2/lib/python2.7/site-packages/pandas/core/generic.py:6586: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


('ACEDEPRS', 24019)
('ACEDRINK', 23929)
('ACEDRUGS', 23848)
('ACEPRISN', 23813)
('ACEDIVRC', 23705)
('ACEPUNCH', 23394)
('ACEHURT', 23302)
('ACESWEAR', 23116)
('ACETOUCH', 22963)
('ACETTHEM', 22914)
('ACEHVSEX', 22883)


In [9]:
#this is the number of people lost by removing all that don't answer with yes or no (remove refuse, unknown, or balnk) for all ACEs
24405-22883

1522

In [10]:
#renidex the cleaned data to make it easier to work with in processing
all_clean = all_clean.reset_index(drop=True)

In [11]:
#look at the cleaned data
all_clean.head()

,@_STATE,@_GEOSTR,@_DENSTR2,PRECALL,REPNUM,REPDEPTH,FMONTH,IDATE,IMONTH,IDAY,...,@_AIDTST2,CPCOUNTY,@_ITSCF1,@_ITSCF2,@_ITSPOST,@_ITSFINL,@_RAWHH,@_WT2HH,@_POSTHH,@_HOUSEWT
0,11,1,1,1,20002,19,2,3012010,3,1,...,1,,455432,127.697,0.516679,65.9785,1,6.87392,9.36442,64.3703
1,11,1,1,1,20003,30,2,3092010,3,9,...,2,,455432,127.697,1.10438,141.026,1,6.87392,9.36442,64.3703
2,11,1,1,1,20005,7,2,3022010,3,2,...,1,,455432,63.8486,0.516679,32.9893,1,6.87392,9.36442,64.3703
3,11,1,1,1,20005,15,2,2272010,2,27,...,,,455432,127.697,0.364598,46.5581,1,6.87392,9.36442,64.3703
4,11,1,2,1,20005,28,2,2282010,2,28,...,2,,455432,95.7847,0.56528,54.1452,1,10.3122,9.36442,96.5673


In [12]:
#trim down to only include the variables I'm interested in
trimmed_data = all_clean[["ACEDEPRS","ACEDRINK","ACEDRUGS","ACEPRISN","ACEDIVRC","ACEPUNCH","ACEHURT","ACESWEAR","ACETOUCH","ACETTHEM","ACEHVSEX",
                        "DIABETE2","CVDINFR4","CVDCRHD4","CVDSTRK3","ASTHMA2","SMOKE100","VETERAN2","ADANXEV","ADDEPEV","@_BMI4CAT","DROCDY2_","@_RFBING4","@_SMOKER3",
                        "GENHLTH","PHYSHLTH","MENTHLTH","QLREST2","EXERANY2",
                        "AGE","EDUCA","INCOME2","SEX","@_PRACE","@_STATE"]]

In [13]:
#could see what it's like if I get rid of all empty spaces mostly out of curisotty could do it with and without anxiety and despression bc that would cut out a whole state
len(trimmed_data)

22883

In [14]:
trimmed_data.head()

,ACEDEPRS,ACEDRINK,ACEDRUGS,ACEPRISN,ACEDIVRC,ACEPUNCH,ACEHURT,ACESWEAR,ACETOUCH,ACETTHEM,...,PHYSHLTH,MENTHLTH,QLREST2,EXERANY2,AGE,EDUCA,INCOME2,SEX,@_PRACE,@_STATE
0,2,2,2,2,2,1,1,3,2,1,...,88,88,5,1,58,6,8,2,1,11
1,1,1,2,1,1,2,3,3,1,1,...,88,88,5,1,35,4,6,2,1,11
2,2,1,2,2,2,1,1,1,1,1,...,1,88,7,1,55,6,8,2,1,11
3,2,1,2,2,2,1,1,3,1,1,...,88,88,88,2,65,6,8,2,1,11
4,2,2,2,2,1,2,1,3,1,1,...,15,88,88,1,58,4,1,2,2,11


In [15]:
#here's the first master data (All ACE stuff is filled in (no 7s, 9s or blank), only includes the variables I'm interested in, indices are correct)
trimmed_data.to_csv("trimmed data_update1.csv")

## Second stage data cleaning

**Edit trimmed data so that:**
- all variables have the correct coding (0 for "no", 1 for "yes") if binary
- remove any "I don't know" or "refuse to answer"
- create dummies where appropriate
- combine or remove answers where needed (covered in the variables spreadsheet)
- remove any variables I have changed my mind on

In [18]:
#to avoid running the first half, just import the data frame created in the last step
trimmed_data = pd.read_csv("/Users/elianasullivan/Documents/Minerva/last one/first semester/capstone/week 7/trimmed data_update1.csv")

/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (13,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [19]:
trimmed_data.head()

,Unnamed: 0,ACEDEPRS,ACEDRINK,ACEDRUGS,ACEPRISN,ACEDIVRC,ACEPUNCH,ACEHURT,ACESWEAR,ACETOUCH,...,PHYSHLTH,MENTHLTH,QLREST2,EXERANY2,AGE,EDUCA,INCOME2,SEX,@_PRACE,@_STATE
0,0,2,2,2,2,2,1,1,3,2,...,88,88,5,1,58,6,8,2,1,11
1,1,1,1,2,1,1,2,3,3,1,...,88,88,5,1,35,4,6,2,1,11
2,2,2,1,2,2,2,1,1,1,1,...,1,88,7,1,55,6,8,2,1,11
3,3,2,1,2,2,2,1,1,3,1,...,88,88,88,2,65,6,8,2,1,11
4,4,2,2,2,2,1,2,1,3,1,...,15,88,88,1,58,4,1,2,2,11


In [20]:
##### CLEAN THE DATA PER ABOVE GUIDELINES
#each variable requires different transformations, they are grouped below

#remove 7s
tranformation_group_1 = ["DIABETE2","CVDINFR4","CVDCRHD4","CVDSTRK3","ASTHMA2","SMOKE100","ADANXEV","ADDEPEV","AGE","VETERAN2","GENHLTH"]
for trait in tranformation_group_1:
    trimmed_data[trait].replace(7, np.nan, inplace=True)
    trimmed_data[trait].replace("7", np.nan, inplace=True)

#remove 9s
tranformation_group_2 = ["DIABETE2","CVDINFR4","CVDCRHD4","CVDSTRK3","ASTHMA2","SMOKE100","ADANXEV","ADDEPEV","AGE","VETERAN2",
                        "@_BMI4CAT","@_RFBING4","EXERANY2", "@_SMOKER3","EDUCA","GENHLTH"]
for trait in tranformation_group_2:
    trimmed_data[trait].replace(9, np.nan, inplace=True)
    trimmed_data[trait].replace("9", np.nan, inplace=True)

#remove 77s and 99s   
tranformation_group_3 = ["PHYSHLTH","MENTHLTH","QLREST2","INCOME2","@_PRACE"]
for trait in tranformation_group_3:
    trimmed_data[trait].replace(77, np.nan, inplace=True)
    trimmed_data[trait].replace(99, np.nan, inplace=True)
    trimmed_data[trait].replace("77", np.nan, inplace=True)
    trimmed_data[trait].replace("99", np.nan, inplace=True)

#convert 2s to 0s
tranformation_group_4 = ["DIABETE2","CVDINFR4","CVDCRHD4","CVDSTRK3","ASTHMA2","SMOKE100","ADANXEV","ADDEPEV",
                        "ACEDEPRS","ACEDRINK","ACEDRUGS","ACEPRISN","ACEDIVRC", "EXERANY2","SEX"]
for trait in tranformation_group_4:
    trimmed_data[trait].replace(2, 0, inplace=True)
    trimmed_data[trait].replace("2", 0, inplace=True)
    
#convert 1s to 0s
tranformation_group_5 = ["ACEPUNCH","ACEHURT","ACESWEAR","ACETOUCH","ACETTHEM","ACEHVSEX","@_BMI4CAT","@_RFBING4"]
for trait in tranformation_group_5:
    trimmed_data[trait].replace(1, 0, inplace=True)
    trimmed_data[trait].replace("1", 0, inplace=True)

#convert and 2s & 3s to 1s
tranformation_group_6 = ["ACEPUNCH","ACEHURT","ACESWEAR","ACETOUCH","ACETTHEM","ACEHVSEX", "@_BMI4CAT","@_SMOKER3", "VETERAN2"]
for trait in tranformation_group_6:
    trimmed_data[trait].replace(2, 1, inplace=True)
    trimmed_data[trait].replace(3, 1, inplace=True)
    trimmed_data[trait].replace("2", 1, inplace=True)
    trimmed_data[trait].replace("3", 1, inplace=True)
    
#convert 88s to 0s
tranformation_group_7 = ["PHYSHLTH","MENTHLTH","QLREST2"]
for trait in tranformation_group_7:
    trimmed_data[trait].replace(88, 0, inplace=True)
    trimmed_data[trait].replace("88", 0, inplace=True) 

#convert 4s to 0s
tranformation_group_8 = ["DIABETE2","@_SMOKER3","VETERAN2"]
for trait in tranformation_group_8:
    trimmed_data[trait].replace(4, 0, inplace=True)
    trimmed_data[trait].replace("4", 0, inplace=True)
    
#FOR INDIVIDUAL TRANSFORMATIONS

#convert 3s to 0s for this outcome
trimmed_data["DIABETE2"].replace(3, 0, inplace=True)
trimmed_data["DIABETE2"].replace("3", 0, inplace=True)

#convert 5s to 0s for this demographic
trimmed_data["VETERAN2"].replace(5, 0, inplace=True)
trimmed_data["VETERAN2"].replace("5", 0, inplace=True)

#remove 900s for this outcome
trimmed_data["DROCDY2_"].replace(900, np.nan, inplace=True)  
trimmed_data["DROCDY2_"].replace("900", np.nan, inplace=True)   

#7s and 8s become 6s for this demographic
trimmed_data["@_PRACE"].replace(7, 6, inplace=True)
trimmed_data["@_PRACE"].replace(8, 6, inplace=True)
trimmed_data["@_PRACE"].replace("7", 6, inplace=True)
trimmed_data["@_PRACE"].replace("8", 6, inplace=True)

#convert 8s to 0s for this ACE
trimmed_data["ACEDIVRC"].replace(8, 0, inplace=True)
trimmed_data["ACEDIVRC"].replace("8", 0, inplace=True)

#convert 2s to 1s for this outcome
trimmed_data["@_RFBING4"].replace(2, 1, inplace=True)
trimmed_data["@_RFBING4"].replace("2", 1, inplace=True)

    
#count np for each column first so I can make sure I won't lose all of the anxiety depression ones by doing this np remove thing (DONE)
#WON'T RUN THIS RIGHT NOW BUT CAN LATER
full_clean = trimmed_data.dropna(axis = 0, how='any')

#make dummies for the ones that actually need it
full_clean = pd.get_dummies(full_clean, columns=["GENHLTH","@_PRACE","@_STATE"])

full_clean = full_clean.reset_index(drop=True)

#just to check how much we have lost total
print(len(full_clean))
print("This stage lost %s observations") % (22883-len(full_clean))

18766
This stage lost 4117 observations


In [21]:
full_clean.head()

,Unnamed: 0,ACEDEPRS,ACEDRINK,ACEDRUGS,ACEPRISN,ACEDIVRC,ACEPUNCH,ACEHURT,ACESWEAR,ACETOUCH,...,@_PRACE_2.0,@_PRACE_3.0,@_PRACE_4.0,@_PRACE_5.0,@_PRACE_6.0,@_STATE_11,@_STATE_15,@_STATE_32,@_STATE_50,@_STATE_55
0,0,0,0,0,0,0,0,0,1,1,...,0,0,0,0,0,1,0,0,0,0
1,1,1,1,0,1,1,1,1,1,0,...,0,0,0,0,0,1,0,0,0,0
2,2,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,3,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
4,4,0,0,0,0,1,1,0,1,0,...,1,0,0,0,0,1,0,0,0,0


In [22]:
#this gets rid of anything empty, which removes a whole states-worth of information
#there are still ~16039 entirely clean which seems worth it for analyses and is also close to CDC numbers
full_clean.replace(empty, np.nan, inplace=True)
full_clean = full_clean.dropna(axis = 0, how='any')
print(len(full_clean))
print("This stage lost %s observations") % (18766-len(full_clean))

16039
This stage lost 2727 observations


In [23]:
full_clean.head()

,Unnamed: 0,ACEDEPRS,ACEDRINK,ACEDRUGS,ACEPRISN,ACEDIVRC,ACEPUNCH,ACEHURT,ACESWEAR,ACETOUCH,...,@_PRACE_2.0,@_PRACE_3.0,@_PRACE_4.0,@_PRACE_5.0,@_PRACE_6.0,@_STATE_11,@_STATE_15,@_STATE_32,@_STATE_50,@_STATE_55
2727,3378,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2728,3379,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0
2729,3380,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2730,3382,0,1,0,0,1,1,0,1,0,...,0,0,1,0,0,0,1,0,0,0
2731,3383,0,0,0,0,0,0,1,1,0,...,0,0,0,0,0,0,1,0,0,0


In [25]:
#list the column names just to check
list(full_clean)

#preview the data frame
full_clean.head()

,Unnamed: 0,ACEDEPRS,ACEDRINK,ACEDRUGS,ACEPRISN,ACEDIVRC,ACEPUNCH,ACEHURT,ACESWEAR,ACETOUCH,...,@_PRACE_2.0,@_PRACE_3.0,@_PRACE_4.0,@_PRACE_5.0,@_PRACE_6.0,@_STATE_11,@_STATE_15,@_STATE_32,@_STATE_50,@_STATE_55
2727,3378,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2728,3379,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0
2729,3380,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2730,3382,0,1,0,0,1,1,0,1,0,...,0,0,1,0,0,0,1,0,0,0
2731,3383,0,0,0,0,0,0,1,1,0,...,0,0,0,0,0,0,1,0,0,0


In [26]:
#export the final clean data to a new csv
full_clean.to_csv("full clean data.csv")